Reading a file with json representing a Gene.
Using JsonSlurper to parse the file, jsonSlurper let's you walk the json structure using dot notation treating the json doc like a map of nested properties.

In [61]:
String jsonText = new File('data/api-export/genes/genes-25.json').text
def slurper = new groovy.json.JsonSlurper()
json =slurper.parseText(jsonText)
OutputCell.HIDDEN

In [62]:
mapByProteinEffect = json.variants.groupBy{it.proteinEffect}
OutputCell.HIDDEN

In [63]:
mapByImpact = json.variants.groupBy{it.impact}
OutputCell.HIDDEN 

In [64]:
def gridLayoutManager = new GridOutputContainerLayoutManager(2)
outputContainer = new OutputContainer()
outputContainer.setLayoutManager(gridLayoutManager)
outputContainer

def proteinEffectList = mapByProteinEffect.keySet() as List
def proteinEffectcounts = mapByProteinEffect.values().collect{it.size()}
def proteinEffectPlot = new CategoryPlot(initWidth: 1000, initHeight: 400, categoryNames:proteinEffectList, title: 'Protein Effect Plot')
proteinEffectPlot << new CategoryBars(value: [proteinEffectcounts])
beakerx.effectMap = [effects : proteinEffectList, counts : proteinEffectcounts]
outputContainer.addItem(proteinEffectPlot, 'Protein Effect Plot')

def impactCategories = mapByImpact.keySet() as List
def impactCounts = mapByImpact.values().collect{it.size()}
/* incoporating beakerX functionalities  */
beakerx.impactMap = [impactCategories : impactCategories, counts : impactCounts]
def impactPlot = new CategoryPlot(initWidth: 1000, initHeight: 400, categoryNames:impactCategories, title:'Impact Plot')
impactPlot << new CategoryBars(value: [impactCounts])

outputContainer.addItem(impactPlot, 'Impact Plot')
outputContainer

# Interactive Bar Charts  plots in Plotly

In [65]:
%%html
<table border='1'>
    <tr>
        <td><div id="plotlyHorizontalBar"/></td><td><div id="plotlyPieChart"/></td>
    </tr>
</table>

,


# More Visualizations

In [66]:
%%javascript
require.config({paths:{'plotly':'//cdn.plot.ly/plotly-latest.min'}});
require(['plotly'], function(Plotly) {
    window.Plotly = Plotly;
});

In [67]:
%%javascript

beakerx.displayHTML(this, '<div id="plotlyHorizontalBar"/>');
var data = [
  {
    x: beakerx.impactMap.counts,
    y: beakerx.impactMap.impactCategories,    
    type: 'bar',
    orientation:'h'
  }
];

Plotly.newPlot('plotlyHorizontalBar', data, {width:500});

In [68]:
%%javascript
beakerx.displayHTML(this, '<div id="plotlyPieChart"></div>');
var data = [{
  values: beakerx.impactMap.counts,
  labels: beakerx.impactMap.impactCategories,
  type: 'pie'
}];

var layout = {
  height: 400,
  width: 500
};

Plotly.newPlot('plotlyPieChart', data, layout);


# Donut  Plots

In [69]:
%%html
<table border='1'>
    <tr>
        <td><div id="plotlyDonutEffect"/></td><td><div id="plotlyDonutImpact"/></td>
    </tr>
</table>

,


# Donut plot categories can be clicked out

In [70]:
%%javascript
beakerx.displayHTML(this, '<div id="plotlyDonutEffect"></div>');
var data = [{
  values: beakerx.impactMap.counts,
  labels: beakerx.impactMap.impactCategories,
  type: 'pie',
  name : 'Impact',
  hoverinfo: 'label+percent+name',
  hole: .4
}];

var layout = {
    title: 'Impact',
    font: {
        family: 'Courier New, monospace',
        size: 12,
        color: '#7f7f7f'
      },
    annotations: {
      
      showarrow: true,
      text: 'Impact',
      textposition: 'inside',
      x: 0.17,
      y: 0.5
    },
    height: 400,
    width: 600,
    showlegend: true
}

Plotly.newPlot('plotlyDonutEffect', data, layout);

In [71]:
%%javascript
beakerx.displayHTML(this, '<div id="plotlyDonutImpact"></div>');
var data = [{
  values: beakerx.effectMap.counts,
  labels: beakerx.effectMap.effects,
  type: 'pie',
  name : 'Impact',
  hoverinfo: 'label+percent+name',
  hole: .4
}];

var layout = {
    title: 'Protein Effect',
    font: {
        family: 'Courier New, monospace',
        size: 12,
        color: '#7f7f7f'
      },
    annotations: {
      font: {
        size: 20
      },
      showarrow: false,
      text: 'Protein Effect',
      textposition: 'inside',
      
      x: 0.17,
      y: 0.5
    },
    height: 400,
    width: 600,
    showlegend: true
}

Plotly.newPlot('plotlyDonutImpact', data, layout);

## Gene Level Evidence Categorical Data With Varied Visuals

In [72]:
%%html
<body>
        <div id="evidenceCategories"/></div>
</body>

In [73]:
// Create multiple maps
mapEvidenceByResponse = json.evidence.groupBy{it.responseType}
mapEvidenceByTherapy = json.evidence.groupBy{it.therapy.therapyName}
mapEvidenceByIndication = json.evidence.groupBy{it.indication.name}
mapEvidenceByStatus = json.evidence.groupBy{it.approvalStatus}
OutputCell.HIDDEN

In [74]:
// map key-value pairs
// keys
def responseList = mapEvidenceByResponse.keySet() as List
def therapyList = mapEvidenceByTherapy.keySet() as List
def indicationList = mapEvidenceByIndication.keySet() as List
def statusList = mapEvidenceByStatus.keySet() as List

// value counts
def responseCounts = mapEvidenceByResponse.values().collect{it.size()}
def therapyCounts = mapEvidenceByTherapy.values().collect{it.size()}
def indicationCounts = mapEvidenceByIndication.values().collect{it.size()}
def statusCounts = mapEvidenceByStatus.values().collect{it.size()}

// map to beakerx
beakerx.responseMap = [response : responseList, counts: responseCounts]
beakerx.therapyMap = [therapy : therapyList, counts : therapyCounts]
beakerx.indicationMap = [indication : indicationList, counts : indicationCounts]
beakerx.statusMap = [status : statusList, counts : statusCounts]

OutputCell.HIDDEN

In [75]:
%%javascript
// plot
beakerx.displayHTML(this, '<div id="evidenceCategories"></div>');

var data = [{
  values: beakerx.responseMap.counts,
  labels: beakerx.responseMap.response,
  domain: {column: 0},
  name: 'Response Type',
  hoverinfo: 'label+percent',
  hole: .6,
  type: 'pie'
},{
  values: beakerx.therapyMap.counts,
  labels: beakerx.therapyMap.therapy,
  domain: {column: 1},
  name: 'Therapy Name',
  hoverinfo: 'label+percent',
  hole: .4,
  type: 'pie'
},
{
    values: beakerx.indicationMap.counts,
    labels: beakerx.indicationMap.indication,
    domain: {column: 2},
    name: 'Indication/ Tumor Type',
    hoverinfo: 'label+percent',
    hole: .565,
    type: 'pie'
            
},
{
    values: beakerx.statusMap.counts,
    labels: beakerx.statusMap.status,
    domain: {column: 3},
    name: 'Approval status',
    hoverinfo: 'label+percent',
    hole: .35,
    type: 'pie'            
}
                      
 ];

var layout = {
  title: 'Gene Level Evidence Categories',
   font: {
        family: 'Courier New, monospace',
        size: 14,
        color: '#7f7f7f'
      },
    annotations: [
    {
      font: {
        size: 14
      },
      showarrow: false,
      text: 'Response',
      x: 0.08,
      y: 0.5
    },
    {
      font: {
        size: 14
      },
      showarrow: false,
      text: 'Therapy',
      x: 0.38,
      y: 0.5
    },
    {
      font: {
        size: 14
      },
      showarrow: false,
      text: 'Indication',
      x: 0.63,
      y: 0.5
    },
    {
      font: {
        size: 14
      },
      showarrow: false,
      text: 'Status',
      x: 0.92,
      y: 0.5
    }  
  ],
  height: 500,
  width: 1800,
  showlegend: true,
  grid: {rows: 1, columns: 4}
};

Plotly.newPlot('evidenceCategories', data, layout);


# Gene Level Evidence Categories SunBurst Chart

In [76]:
%%html
<body>
        <div id="GeneSunburst"/></div>
</body>

In [77]:
%%javascript
// plot
beakerx.displayHTML(this, '<div id="GeneSunburst"></div>');

// added inputs manually, need to figure out way to map gene level evidence data into treemap, then build sunburst chart
var data = [{
  type: "sunburst",
  labels: ["Gene Level Evidence", "Tumor Type", "Response Type", "Therapy Name", "Approval Status", "Evidence Type", "Oral cavity cancer", "Lung adenocarcinoma", "Dasatinib","Nilotinib","Imatinib", "Preclinical", "Case Report/Case Series","Preclinical Cell line xenograft"],
  parents: ["", "Gene Level Evidence","Gene Level Evidence","Gene Level Evidence","Gene Level Evidence","Gene Level Evidence", "Tumor Type", "Tumor Type", "Therapy Name", "Therapy Name","Therapy Name", "Approval Status","Approval Status","Approval Status"],
  values:  [10,7,7,7,7,7, 1, 6, 3,1,3,4,1,2],
  branchvalues: "remainder",
  outsidetextfont: {size: 20, color: "#377eb8"},
  leaf: {opacity: 0.4},
  marker: {line: {width: 2}},
}];

var layout = {
  margin: {l: 0, r: 0, b: 0, t: 0},
  width: 1000,
  height: 800,
    showlegend: true
};


Plotly.newPlot('GeneSunburst', data, layout);

## Disease Ontology Sunburst

### Efficacy Evidence data sunburst

In [78]:
%%html
<body>
        <div id="EESunburst"/></div>
</body>

In [79]:
String jsonText = new File('data/IndicationEEData.json').text
def slurper = new groovy.json.JsonSlurper()
json =slurper.parseText(jsonText)
OutputCell.HIDDEN

In [80]:
// get map
EElabels = json.labels
EEparents = json.parentsLabels
EEvalues = json.values
println("**\n"+EElabels)
println("**\n"+EEparents)
println("**\n"+EEvalues)

beakerx.EElabels = EElabels
beakerx.EEparents = EEparents
beakerx.EEvalues = EEvalues

OutputCell.HIDDEN

**


[disease, disease of cellular proliferation, cancer, organ system cancer, reproductive organ cancer, female reproductive organ cancer, vaginal cancer, vaginal squamous tumor, endocrine gland cancer, neuroendocrine tumor, musculoskeletal system cancer, connective tissue cancer, bone cancer, gastrointestinal system cancer, intestinal cancer, large intestine cancer, colorectal cancer, colon cancer, colon carcinoma, colon adenocarcinoma, benign neoplasm, organ system benign neoplasm, cardiovascular organ benign neoplasm, hemangioma, urinary system cancer, kidney cancer, hematologic cancer, leukemia, lymphocytic leukemia, chronic lymphocytic leukemia, hairy cell leukemia, cell type cancer, carcinoma, adenocarcinoma, giant cell tumor, tenosynovial giant cell tumor, uterine cancer, nervous system cancer, central nervous system cancer, cranial nerve neoplasm, cranial nerve malignant neoplasm, olfactory nerve neoplasm, olfactory neuroblastoma, adrenal gland cancer, adrenal cortex cancer, hepato

**


[, disease, disease of cellular proliferation, cancer, organ system cancer, reproductive organ cancer, female reproductive organ cancer, vaginal cancer, organ system cancer, endocrine gland cancer, organ system cancer, musculoskeletal system cancer, connective tissue cancer, organ system cancer, gastrointestinal system cancer, intestinal cancer, large intestine cancer, colorectal cancer, colon cancer, colon carcinoma, disease of cellular proliferation, benign neoplasm, organ system benign neoplasm, cardiovascular organ benign neoplasm, organ system cancer, urinary system cancer, organ system cancer, hematologic cancer, leukemia, lymphocytic leukemia, chronic lymphocytic leukemia, cancer, cell type cancer, carcinoma, connective tissue cancer, giant cell tumor, female reproductive organ cancer, organ system cancer, nervous system cancer, central nervous system cancer, cranial nerve neoplasm, cranial nerve malignant neoplasm, olfactory nerve neoplasm, endocrine gland cancer, adrenal gland

**
[0, 0, 76, 0, 0, 6, 0, 1, 1, 15, 0, 0, 2, 15, 1, 1, 901, 270, 41, 20, 0, 0, 0, 1, 4, 12, 187, 111, 3, 66, 6, 0, 5, 1, 0, 2, 10, 0, 4, 0, 0, 0, 2, 3, 1, 0, 17, 0, 116, 2, 61, 25, 27, 1, 0, 0, 0, 0, 8, 0, 1, 135, 0, 1, 1, 5, 20, 5, 49, 1, 0, 299, 131, 16, 1, 0, 763, 6, 8, 1, 71, 1, 3, 11, 1, 210, 4, 4, 1, 0, 0, 6, 0, 5, 1088, 6, 0, 2, 16, 38, 1, 1, 0, 2, 0, 5, 245, 13, 1, 11, 23, 10, 7, 0, 3, 1, 24, 0, 290, 12, 10, 1, 1, 0, 1, 0, 2, 53, 17, 6, 29, 5, 7, 3, 2, 2, 17, 0, 2, 11, 218, 58, 9, 6, 3, 1, 23, 4, 4, 5, 7, 0, 0, 5, 12, 0, 21, 1, 0, 2, 8, 0, 5, 59, 1, 9, 50, 3, 6, 0, 25, 0, 0, 1, 11, 1, 9, 36, 88, 4, 1, 4, 4, 4, 0, 9, 1, 1, 2, 0, 52, 2, 201, 26, 49, 2, 20, 0, 92, 1, 24, 1307, 89, 349, 3, 1, 13, 2, 6, 21, 61, 155, 11, 24, 3, 5, 0, 0, 0, 4, 1, 1, 0, 6, 2, 1, 1, 8, 10, 1, 2, 7, 0, 1, 1, 9, 111, 3, 8, 47, 2, 1, 1, 1, 0, 1, 1, 9, 1, 4, 2, 4, 9, 4, 3, 1, 0, 1, 1, 1, 1, 1, 0, 1, 13, 1, 1, 0, 0, 0, 34, 52, 1, 0, 0, 1, 1, 2, 39, 9, 24, 10, 1, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 1, 0, 2, 1, 8, 1

In [81]:
println("label size :"+beakerx.EElabels.size())
println("parent size :"+beakerx.EEparents.size())
println("values size :"+beakerx.EEvalues.size())
OutputCell.HIDDEN

label size :463
parent size :463
values size :463


In [82]:
%%javascript
// plot
beakerx.displayHTML(this, '<div id="EESunburst"></div>');

// added inputs manually, need to figure out way to map gene level evidence data into treemap, then build sunburst chart
var data = [{
  type: "sunburst",
  labels: beakerx.EElabels,
  parents:beakerx.EEparents,
  values: beakerx.EEvalues,
  //branchvalues: 'total',
  outsidetextfont: {size: 20, color: "#377eb8"},
  leaf: {opacity: 0.4},
  marker: {line: {width: 2}},
  
}];

var layout = {
  margin: {l: 0, r: 0, b: 0, t: 0},
  width: 1000,
  height: 800,
    showlegend: true
};


Plotly.newPlot('EESunburst', data, layout);

### Cinical Trial  data sunburst

In [83]:
String jsonText = new File('data/IndicationClinicalTrialData.json').text
def slurper = new groovy.json.JsonSlurper()
json =slurper.parseText(jsonText)
OutputCell.HIDDEN

### pie chart

In [84]:
%%html
<body>
        <div id="CTpiechart"/></div>
</body>

In [85]:
// get map
CTlabels = json.labels
CTparents = json.parentsLabels
CTvalues = json.values
// println("**\n"+CTlabels)
// println("**\n"+CTparents)
// println("**\n"+CTvalues)

beakerx.CTlabels =CTlabels
beakerx.CTparents = CTparents
beakerx.CTvalues = CTvalues

println(CTlabels[289..292])
println(CTparents[289..292].collect{"'${it}'"})

[adamantinous craniopharyngioma, papillary craniopharyngioma, infratentorial cancer, medulloblastoma]
['craniopharyngioma', 'craniopharyngioma', 'brain cancer', 'infratentorial cancer']


null

In [86]:
println("Quality Check\n**\n")
println("label size :"+beakerx.CTlabels.size())
println("parent size :"+beakerx.CTparents.size())
println("values size :"+beakerx.CTvalues.size())
OutputCell.HIDDEN

Quality Check
**

label size :700
parent size :700
values size :700


In [87]:
%%javascript
beakerx.displayHTML(this, '<div id="CTpiechart"></div>');
var data = [{
  values: beakerx.CTvalues,
  labels: beakerx.CTlabels,
  type: 'pie',
  name : 'Impact',
  hoverinfo: 'label+percent+name',
  hole: .4
}];

var layout = {
    title: 'Clinical Trial',
    font: {
        family: 'Courier New, monospace',
        size: 12,
        color: '#7f7f7f'
      },
    annotations: {
      
      showarrow: true,
      text: 'Clinical Trial',
      textposition: 'inside',
      x: 0.5,
      y: 0.5
    },
    height: 800,
    width: 1000,
    showlegend: false
}

Plotly.newPlot('CTpiechart', data, layout);

## sunburst

In [88]:
%%html
<body>
        <div id="CTsunburst"/></div>
</body>

In [89]:
%%javascript


In [90]:
%%javascript
// plot
beakerx.displayHTML(this, '<div id="CTSunburst"></div>');
var data = [{
  type: "sunburst",
  //maxdepth: 3, // wrong
  // issue with map relations on data, after 291, no visual  
  labels: beakerx.CTlabels,
  parents: beakerx.CTparents,
  values: beakerx.CTvalues,
  outsidetextfont: {size: 20, color: "#377eb8"},
  textinfo : 'label',
  //opacity : 0.5, // opacity of whole chart
  leaf: {opacity: 0.4},
  marker: {line: {width: 2}},
}];

var layout = {
  margin: {l: 0, r: 0, b: 0, t: 0},
  width: 1000,
  height: 800,
  showlegend: false
};


Plotly.newPlot('CTSunburst', data, layout);

## More features

In [91]:
String jsonText = new File('data/IndicationClinicalTrialData_multiValue.json').text
def slurper = new groovy.json.JsonSlurper()
json =slurper.parseText(jsonText)
OutputCell.HIDDEN

In [92]:
// get map
CTlabels = json.labels
CTparents = json.parentsLabels
CTvalues = json.values
CTVdescendants = json.descendantValues
 println("**\n"+CTlabels)
// println("**\n"+CTparents)
// println("**\n"+CTvalues)
//println("**\n"+CTVdescendants)

beakerx.CTVlabels =CTlabels
beakerx.CTVparents = CTparents
beakerx.CTVvalues = CTvalues
beakerx.CTVdescendants = CTVdescendants
//println(CTlabels[289..292])
//println(CTparents[289..292].collect{"'${it}'"})

OutputCell.HIDDEN

**


[disease, disease of cellular proliferation, cancer, organ system cancer, reproductive organ cancer, female reproductive organ cancer, vaginal cancer, vaginal squamous tumor, endocrine gland cancer, neuroendocrine tumor, nervous system cancer, neuroectodermal tumor, cardiovascular cancer, vascular cancer, musculoskeletal system cancer, connective tissue cancer, bone cancer, sex cord-gonadal stromal tumor, giant cell tumor, gastrointestinal system cancer, intestinal cancer, large intestine cancer, colorectal cancer, colon cancer, colon carcinoma, colon adenocarcinoma, benign neoplasm, organ system benign neoplasm, cardiovascular organ benign neoplasm, hemangioma, urinary system cancer, kidney cancer, hemangiopericytoma, hematologic cancer, leukemia, lymphocytic leukemia, chronic lymphocytic leukemia, hairy cell leukemia, cell type cancer, carcinoma, adenocarcinoma, tenosynovial giant cell tumor, disease of anatomical entity, hematopoietic system disease, leukocyte disease, mastocytosis,

In [93]:
%%python

# uncomment if running from Conda
from beakerx.object import beakerx

t = []
for val in beakerx.CTVdescendants:
    t.append("child sum: "+str(val))

    v = []    
for val in beakerx.CTVvalues:
    v.append("branch remainder: "+str(val))
    
combined = []
stri = ""
for i in range(len(beakerx.CTVvalues)):
    combined.append(v[i]+" \n "+t[i]);

#print ("t: ",t)
#print ("v: ",v)
#print ("combined :",combined)

beakerx.remval = v
beakerx.textsum = t
beakerx.comb = combined

In [105]:
%%python

# uncomment if running from Conda
from beakerx.object import beakerx

l= []

maxval = max(beakerx.CTVvalues)
minval = min(beakerx.CTVvalues)



# percentage colorscale from min to max
"""
# uncomment code below to test
for value  in beakerx.CTVvalues:
    r =0
    g=0
    b=0
    
    perc = (value/maxval)*100
    
    if(perc <50):
        r=255
        g= round(5.1*perc)
    else:
        g=255
        r=round(510-5.10*perc)
    
    h = (r*0x10000)+(g*0x100)+(b*0x1)
    #print ("h :",h)
    
    l.append("#"+hex(h).lstrip("0x").rstrip("L"))
"""

ls = []
# 10 categories for colors
colorslist = ['#ED8B00', '#333F48', '#A2AAAD', '#0085CA', '#9467bd', '#43B02A', '#002D72', '#7f7f7f', '#bcbd22', '#D0D3D4']
#colorslist =['#0085CA', '#002D72', '#2C2A29', '#333F48', '#A2AAAD', '#D0D3D4', '#FFFFFF','#F1C400', '#ED8B00', '#DC582A', '#AB2328', '#8A1B61', '#005151', '#43B02A']
#['#ED8B00', '#333F48', '#A2AAAD', '#0085CA', '#9467bd', '#43B02A', '#002D72', '#7f7f7f', '#bcbd22', '#17becf', '#D0D3D4']
#print ("length : ",len(colorslist), "zeroth :", colorslist[0])
frac = maxval/9
#print ("frac :",frac)
for value  in beakerx.CTVvalues:
    ind = round(float(value/frac))
    
    #print ("ind :", ind)
    
    ls.append( colorslist[ind])

# second set of values based off     
    
lt =[]
maxval = max(beakerx.CTVdescendants)
minval = min(beakerx.CTVdescendants)
tenth = maxval/9

for value  in beakerx.CTVdescendants:
    ind = round(value/tenth)
    #print("ind :", index)
    
    lt.append(colorslist[ind])

#print ("l :",l)
    
beakerx.cols= l
beakerx.vcols = ls
beakerx.tcols = lt

In [106]:
%%javascript

// plot


var data = [{
  type: "sunburst",
  //maxdepth: 3, // wrong
  labels: beakerx.CTVlabels,
  parents: beakerx.CTVparents,
  values: beakerx.CTVvalues,
  hoverinfo : "label+value+text",
  textinfo : 'label',
  opacity: 0.9,
  customdata: beakerx.CTVdescendants,
  
  //hovertext : beakerx.textsum,
  
    //template
  hovertemplate:'<b>Category</b>: %{label}<br>'  +
                '<b>Category Remainder</b>: %{value}<br>' +
                '<b>Child Categories Sum</b>: %{customdata}', 
  
  outsidetextfont: {size: 20, color: "#377eb8"},
  leaf: {opacity: 1},
  marker: {line: {width: 2},
           colors: beakerx.vcols},
}];

var layout = {
  margin: {l: 0, r: 0, b: 0, t: 0},
  width: 1000,
  height: 800,
  showlegend: false
};

var updatemenus=[
    {
        buttons: [
            {
                args: ['type', 'surface'],
                label: '3D Surface',
                method: 'restyle'
            },
            {
                args: ['type', 'heatmap'],
                label:'Heatmap',
                method:'restyle'
            }
        ],
        direction: 'left',
        pad: {'r': 10, 't': 10},
        showactive: true,
        type: 'buttons',
        x: 0.1,
        xanchor: 'left',
        y: 1.1,
        yanchor: 'top'
    }
]



beakerx.data2 = data;
beakerx.layout = layout;

console.log(beakerx.cols);

function myFunction() {
    // replot sunburst
  Plotly.newPlot('CTOSunburst', data, layout);

}

In [107]:
%%javascript
beakerx.displayHTML(this, '<div id="CTOSunburst"></div>');
Plotly.newPlot('CTOSunburst', beakerx.data2, beakerx.layout);

In [ ]:
%%javascript

// plot


var data = [{
  type: "sunburst",
  //maxdepth: 3, // wrong
  labels: beakerx.CTVlabels,
  parents: beakerx.CTVparents,
  values: beakerx.CTVvalues,
  hoverinfo : "label+value+text", 
  textinfo : 'label',
  opacity: 0.9,
  customdata: beakerx.CTVdescendants,
  
  hovertext : beakerx.textsum,
  // template overides hoverinfo
  hovertemplate: '<b>Category</b>: %{label}<br>'  +
                        '<b>Category Remainder</b>: %{value}<br>' +
                 '<b>Child Categories Sum</b>: %{customdata}',
  outsidetextfont: {size: 20, color: "#377eb8"},
  leaf: {opacity: 1},
  marker: {line: {width: 2},
           colors: beakerx.tcols},
}];

var layout = {
  margin: {l: 0, r: 0, b: 0, t: 0},
  width: 1000,
  height: 800,
  showlegend: false
};


beakerx.data2 = data;
beakerx.layout = layout;

console.log(beakerx.cols);

// replot sunburst
beakerx.displayHTML(this, '<div id="CTO2Sunburst"></div>');
Plotly.newPlot('CTO2Sunburst', beakerx.data2, beakerx.layout);


In [98]:
%%html

<body>  
    
    <div id="CTOsunburst"/></div>
    <button onclick="myFunction()">Reset</button>  
</body>

Reset

In [ ]:
%%html

<body>  
    
    <div id="CTO2sunburst"/></div>
    <button onclick="myFunction()">Reset</button>  
</body>

